<img src="Archivos/miad4.png" width=900x>

 # TAREA 2

**Departamento de Ingeniería Industrial.**

**Maestría en Inteligencia Analítica para la Toma de Decisiones (Analytics).**

**Modelos de Análisis Estadístico (MIIA 4100) - 202320.**

Por ningún motivo se recibirán entregas posteriores a la hora límite. En caso de no realizar la entrega la nota asignada será 0.

Notas: La entrega del taller deberá hacerse en un informe autocontenido incluyendo las **gráficas,interpretación y conclusiones**. Adicionalmente debe entregarse el código utilizado dentro de este mismo notebook, de considerar necesario comentar todas las partes del codigo que crean necesarias para el entendimiento del codigo.

Para todos los literales que lo requieran, se recomienda utilizar una significancia del 5% (confianza del 95%). En caso de utilizar otro nivel de significancia, hacerlo explícito en el informe.

______________________________________________________________________



In [30]:
#Librerias que podria necesitar
import pandas as pd
import matplotlib.pyplot as plt


## Ejercicio 1
(35%) La base de datos contenida en diamantes.csv tiene información de los precios 53.940 diamantes teniendo en cuenta los carates, el corte, el color, la claridad, la profundidad, la longitud (x), el ancho (y) y la profundidad (z). <br>


a. (10%) Teniendo en cuenta los datos suministrados: ¿De qué variable o variables depende el valor de los diamantes?

In [1]:
# Implementa tu respuesta en esta celda



b. (10%) Realice un modelo de regresión lineal múltiple para el precio de cada diamante teniendo en cuenta la longitud, el ancho y la profundidad de los diamantes, ponga su ecuación y estime los valores de los diamantes para los siguientes tamaños:
|<center>X</center>|<center>Y</center>|<center>Z</center>|
|-----------|-----------|-----------|
|3.10|<center>3.17</center>| 3.43|
|4.14|<center>3.25</center>|3.11|
|4.00|<center>4.45</center>|3.14|
|4.22|<center>4.14</center>|4.15|
|4.14|<center>2.35</center>|2.85|
|2.94|<center>4.96</center>|4.12|
|4.95|<center>5.10</center>|3.47|
|4.17|<center>4.01</center>|5.13|
|2.77|<center>3.65</center>|5.17|
|4.15|<center>3.05</center>|3.18|
|4.16|<center>3.15</center>|4.17|
|3.90|<center>4.12</center>|3.16|

¿Qué puede concluir de los resultados obtenidos?

In [3]:
# Implementa tu respuesta en esta celda



c. (10%) Realice un modelo de regresión lineal simple para el precio de cada diamante teniendo en cuenta los carates de los diamantes, ponga su ecuación. Responda las siguientes preguntas: <br>

- ¿Es cierta la hipótesis que afirma que la los carates no influyen en el valor de los diamantes?
- Hallé el coeficiente de correlación.
- ¿Qué puede decir de la homocedasticidad? Justifique y grafique. 



In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



## Ejercicio 2

(30%) 30%) Teniendo en cuenta la base de datos contenida en taxis.cvs, donde se muestran los datos de 6433 carreras de taxis, con su fecha de recogida, de bajada de los pasajeros, el número de pasajeros, las distancias, las tarifas, las propinas, el método de pago, la zona de recogida y de bajada.  <br>

a. (15%) Realice un modelo de regresión lineal simple para el precio de cada carrera teniendo en cuenta el tiempo que esta se demoró en minutos, ponga su ecuación. Responda las siguientes preguntas:
- ¿Es cierta la hipótesis que afirma que el tiempo no influye en el valor de la carrera?
- Hallé el coeficiente de correlación.
- ¿Qué puede decir de la homocedasticidad? Justifique y grafique.


In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



b. (15%) Realice un modelo de regresión lineal simple para el precio de cada carrera teniendo en cuenta la distancia, ponga su ecuación. Responda las siguientes preguntas:
- ¿Es cierta la hipótesis que afirma que la distancia influye en el valor de la carrera?
- Hallé el coeficiente de correlación.
- ¿Qué puede decir de la homocedasticidad? Justifique y grafique.


In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



In [ ]:
# Implementa tu respuesta en esta celda



## Ejercicio 3

(35%) De acuerdo con la base de datos de Boston.cvs, en la cual se muestran el valor promedio de 506 predios en $1000’s y las siguientes variables:

|<center>Index</center>|<center>Meaning</center>|
|-----------|-----------|
|<center> 1. CRIM|<center>per capita crime rate by town</center>|
|<center> 2. ZN|<center>proportion of residential land zoned for lots over 25,000 sq.ft.</center>|
|<center> 3. INDUS|<center>proportion of non-retail business acres per town</center>|
|<center> 4. CHAS|<center>Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)</center>|
|<center> 5. NOX|<center>nitric oxides concentration (parts per 10 million)</center>|
|<center> 6. RM|<center>average number of rooms per dwelling</center>|
|<center> 7. AGE|<center>proportion of owner-occupied units built prior to 1940</center>|
|<center> 8. DIS|<center>weighted distances to five Boston employment centres</center>|
|<center> 9. RAD|<center>index of accessibility to radial highways</center>|
|<center> 10. TAX|<center>full-value property-tax rate per 10,000</center>|
| 11. PTRATIO|<center>pupil-teacher ratio by town</center>|
|<center> 12. B|<center>1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town</center>|
|<center> 13. LSTAT|<center>% lower status of the population</center>|
|<center> 14. MEDV|<center>Median value of owner-occupied homes in $1000's</center>|

b. (10%)  Determine las caracteristicas mas influyentes en el precio de las viviendas de Boston. Realice los analisis que considere relevantes como analisis de correlación o graficas de donde se interpreten los resultados.

In [ ]:
# Implementa tu respuesta en esta celda



a. (25%) Con el conjunto de datos suministrado de Boston, realizen un modelo de regresion múltiple que prediga el precio medio de las viviendas en diferentes áreas en función de diversas características. Si considera necesario experimentar con diferentes configuraciones adelante.

In [ ]:
# Implementa tu respuesta en esta celda



## Créditos

__Autores__: Laura Valentina Diaz Pinzon

__Actualizado por__: Alejandra Tabares Pozos. 

__Fecha última actualización__: 01/09/2023